In [1]:
import pathlib
import tensorflow as tf

train_dir = pathlib.Path('DRAM_train/')
test_dir = pathlib.Path('DRAM_test/')

m = 224
batch = 32

In [2]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    seed=123,
    image_size=(m, m),
    batch_size=batch
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    seed=123,
    image_size=(m, m),
    batch_size=batch
)
num_classes = 4

Found 5677 files belonging to 4 classes.
Found 583 files belonging to 4 classes.


In [3]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
base_model = tf.keras.applications.ResNet50(include_top=False, pooling='avg', weights=None, input_shape=(m, m, 3))

x = base_model.output
predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

for layer in base_model.layers:
    layer.trainable = False


model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

model.compile(
  optimizer=tf.optimizers.Adam(learning_rate=1e-10),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=12
)

Epoch 1/12
 64/710 [=>............................] - ETA: 20:06 - loss: 6.7357 - accuracy: 0.2773

In [ ]:
model.summary()